In [42]:
# Libraries
import pandas as pd
import plotly.express as px
from datetime import datetime
from darts import TimeSeries
import plotly.graph_objs as go
from darts.models import ExponentialSmoothing

In [26]:
df=pd.read_csv('sensor_data.csv')
df.head()

,time,SensorA,SensorB,SensorC
0,00:00,1.416250,4.219930,3.139646
1,00:01,3.534439,NaN,3.064088
2,00:02,5.659733,NaN,2.925565
3,00:03,5.640167,3.073650,5.381210
4,00:04,4.454474,1.554044,1.055965


We see this start at midnight and ends 24h later.

In [27]:
df.tail()

,time,SensorA,SensorB,SensorC
1435,23:55,8.417096,4.686364,9.014305
1436,23:56,12.413407,5.709658,7.005713
1437,23:57,11.400624,7.063344,7.522029
1438,23:58,9.707463,7.903916,NaN
1439,23:59,11.220124,8.270642,6.609032


In [28]:
#Overview
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   time     1440 non-null   object 
 1   SensorA  1303 non-null   float64
 2   SensorB  1205 non-null   float64
 3   SensorC  1392 non-null   float64
dtypes: float64(3), object(1)
memory usage: 45.1+ KB


We can see some missing values in each sensor columns.
We also have a time column.
We can interpret this dataset in different ways. We do not have so much information so we can either say all the sensors are on the same spot and we can do the average of the 3. Or we can treat them individually assuming they are placed on different spots.

From the graph we can see the diffrence in measurements for the same time. So We hope those sensors are not based on the same spot...

So We will treat only the sensorA.

In [29]:
df.describe()

,SensorA,SensorB,SensorC
count,1303.000000,1205.000000,1392.000000
mean,9.793641,3.215945,8.013604
std,2.038163,2.843696,2.138642
min,1.416250,-2.740000,1.055965
25%,8.466171,1.114624,6.524029
50%,9.809673,3.210922,8.170571
75%,11.238712,5.464741,9.618772
max,18.107463,9.564000,14.307993


To deal with missing values, replacing by the median or mean doest not seem to be a good option. moving average or mean between all the sensors for the same time...maybe..


In [30]:
sensorA= df.iloc[:,0:2]
sensorA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   time     1440 non-null   object 
 1   SensorA  1303 non-null   float64
dtypes: float64(1), object(1)
memory usage: 22.6+ KB


In [31]:
sensorA['time']=pd.to_datetime(sensorA['time'])
sensorA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   time     1440 non-null   datetime64[ns]
 1   SensorA  1303 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 22.6 KB


/tmp/ipykernel_6764/2918355234.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [32]:
sensorA

,time,SensorA
0,2024-05-24 00:00:00,1.416250
1,2024-05-24 00:01:00,3.534439
2,2024-05-24 00:02:00,5.659733
3,2024-05-24 00:03:00,5.640167
4,2024-05-24 00:04:00,4.454474
...,...,...
1435,2024-05-24 23:55:00,8.417096
1436,2024-05-24 23:56:00,12.413407
1437,2024-05-24 23:57:00,11.400624
1438,2024-05-24 23:58:00,9.707463


In [33]:
sensorA=sensorA.set_index(pd.DatetimeIndex(sensorA['time'])).drop(columns='time')
sensorA.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1440 entries, 2024-05-24 00:00:00 to 2024-05-24 23:59:00
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SensorA  1303 non-null   float64
dtypes: float64(1)
memory usage: 22.5 KB


In [34]:
sensorA

,SensorA
time,
2024-05-24 00:00:00,1.416250
2024-05-24 00:01:00,3.534439
2024-05-24 00:02:00,5.659733
2024-05-24 00:03:00,5.640167
2024-05-24 00:04:00,4.454474
...,...
2024-05-24 23:55:00,8.417096
2024-05-24 23:56:00,12.413407
2024-05-24 23:57:00,11.400624


# Resample

We are resampling the datas doing the mean of the 5 min before the label including the label in the mean.

In [35]:
sensorA_res=sensorA.resample('5min',label='right',closed='right').mean()
sensorA_res

,SensorA
time,
2024-05-24 00:00:00,1.416250
2024-05-24 00:05:00,5.033728
2024-05-24 00:10:00,5.341291
2024-05-24 00:15:00,4.644082
2024-05-24 00:20:00,7.629764
...,...
2024-05-24 23:40:00,11.322034
2024-05-24 23:45:00,9.987526
2024-05-24 23:50:00,11.059341


In [36]:
sensorA_res.info()
#No missing values.

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 289 entries, 2024-05-24 00:00:00 to 2024-05-25 00:00:00
Freq: 5min
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SensorA  289 non-null    float64
dtypes: float64(1)
memory usage: 4.5 KB


In [37]:
fig1=px.line(sensorA_res, y='SensorA')
fig1.show()

We can see an initial increase from 0.00 to 0.50. Then it stay more or less constant and we see another big increase at 17.15.

Let's resample by hour.


In [38]:
sensorA_60res=sensorA.resample('60min',label='right',closed='right').mean()
sensorA_60res

,SensorA
time,
2024-05-24 00:00:00,1.416250
2024-05-24 01:00:00,7.832740
2024-05-24 02:00:00,9.582315
2024-05-24 03:00:00,10.339462
2024-05-24 04:00:00,10.362713
2024-05-24 05:00:00,9.910676
2024-05-24 06:00:00,10.179804
2024-05-24 07:00:00,9.771181
2024-05-24 08:00:00,9.684664


In [40]:
fig2=px.line(sensorA_60res,y='SensorA')
fig2.show()

# Three sensors together

In [45]:
df

,time,SensorA,SensorB,SensorC
0,00:00,1.416250,4.219930,3.139646
1,00:01,3.534439,NaN,3.064088
2,00:02,5.659733,NaN,2.925565
3,00:03,5.640167,3.073650,5.381210
4,00:04,4.454474,1.554044,1.055965
...,...,...,...,...
1435,23:55,8.417096,4.686364,9.014305
1436,23:56,12.413407,5.709658,7.005713
1437,23:57,11.400624,7.063344,7.522029
1438,23:58,9.707463,7.903916,NaN


In [46]:
df=df.set_index(pd.DatetimeIndex(df['time'])).drop(columns='time')
df_res=df.resample('5min',label='right',closed='right').mean()
df_60res=df.resample('60min',label='right',closed='right').mean()
fig3=px.line(df_60res)
fig3.show()

In [47]:

fig4=px.line(df_res)
fig4.show()

In [48]:
fig4=px.box(df)
fig4.show()

# Conclusion

In this small project we have use
- venv
- git
- datetime series


